In [4]:
from scipy import sparse
from sklearn import linear_model
from collections import Counter
import numpy as np
import operator
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
import math
from scipy.stats import norm
import pandas as pd
import sklearn
import tensorflow as tf
from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.feature_selection import chi2

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
sns.set_style("whitegrid")
import altair as alt

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import WhiteKernel

from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import ShuffleSplit

# Code for hiding seaborn warnings
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import nltk
from collections import Counter

#Sets random seeds for reproducibility
seed=159259
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import PredefinedSplit
!pip install hypopt
from hypopt import GridSearch

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [5]:
#Majority class	| BoW L2 LogReg
#0.36	| 0.37

Some paths in the next 3 cells below (as well as a cell later importing glove embeddings) may need some changes to its path. The data files and glove embeddings file are also needed.

In [6]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
service = build('drive', 'v3')

In [7]:
# Run this to mount the Notebook in your Google Drive account 
from google.colab import drive
#drive.mount('/content/drive')
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [8]:
train_path = "/content/drive/My Drive/INFO 159/AP4/train.txt"
test_path = "/content/drive/My Drive/INFO 159/AP4/test.txt"
dev_path = "/content/drive/My Drive/INFO 159/AP4/dev.txt"

train = pd.read_table(train_path, names = ['num', 'score', 'text'])
test = pd.read_table(test_path, names = ['num', 'score', 'text'])
dev = pd.read_table(dev_path, names = ['num', 'score', 'text'])

In [9]:
df = pd.concat([train, dev, test], axis=0)

In [11]:
df.head()

,num,score,text
0,417,2,Which fairground ride is made of iron? The fer...
1,495,3,Why does a chicken coupe only have two doors? ...
2,378,1,Where do you go to weigh a pie? Somewhere over...
3,670,1,Choose a major you love and you won't have to ...
4,345,4,What animal is best at hitting a baseball? A bat!


In [12]:
# Copied from https://github.com/dbamman/nlp22/blob/main/AP/OrdinalRegression.ipynb

Creating some features that may be useful for our problem. 

In [13]:
def featurize(df):
  df["sentence_length"] = df["text"].apply(lambda sent: len(sent))
  df["contain_?"] = df["text"].apply(lambda sent: int("?" in sent))
  df["contain_!"] = df["text"].apply(lambda sent: int("!" in sent))
  df["first_?"] = df["text"].apply(lambda sent: sent.find('?'))
  df["prop_?"] = df["text"].apply(lambda sent: sent.find('?') / len(sent))

  tokenized_text = df["text"].apply(lambda sent: nltk.word_tokenize(sent.lower()))
  #df["first_word"] = tokenized_text.apply(lambda l: l[0])
  df["token_length"] = tokenized_text.apply(lambda sent: len(sent))

  sia = SentimentIntensityAnalyzer()
  sia_dict = df["text"].apply(lambda sent: sia.polarity_scores(sent))
  df["sia_positive"] = sia_dict.apply(lambda d: d["pos"])
  df["sia_negative"] = sia_dict.apply(lambda d: d["neg"])
  df["sia_neutral"] = sia_dict.apply(lambda d: d["neu"])
  df["sia_compound"] = sia_dict.apply(lambda d: d["compound"])

  #df["tfidf_transform"] = df["text"].apply(lambda sent: sent.TfidfTransformer(norm=None))
  vect = CountVectorizer()
  X = vect.fit_transform(df["text"])
  count_vect_df = pd.DataFrame(X.todense(), columns=vect.get_feature_names())
  df.reset_index(inplace=True, drop=True)
  df = pd.concat([df, count_vect_df], axis=1)

  return df

In [48]:
featurize(df).head()

,num,score,text,sentence_length,contain_?,contain_!,first_?,prop_?,token_length,sia_positive,...,yy,zelda,zero,zeroes,zippo,zoidberg,zombie,zombies,zoo,zuckerberg
0,417,2,Which fairground ride is made of iron? The fer...,56,1,0,37,0.660714,11,0.000,...,0,0,0,0,0,0,0,0,0,0
1,495,3,Why does a chicken coupe only have two doors? ...,85,1,1,44,0.517647,21,0.000,...,0,0,0,0,0,0,0,0,0,0
2,378,1,Where do you go to weigh a pie? Somewhere over...,97,1,0,30,0.309278,24,0.128,...,0,0,0,0,0,0,0,0,0,0
3,670,1,Choose a major you love and you won't have to ...,131,0,0,-1,-0.007634,30,0.143,...,0,0,0,0,0,0,0,0,0,0
4,345,4,What animal is best at hitting a baseball? A bat!,49,1,1,41,0.836735,12,0.391,...,0,0,0,0,0,0,0,0,0,0


In [14]:
def xy_split(df):
  return df.iloc[:, 3:], df.iloc[:, 1]

def train_test_dev_split(df):
  return df.iloc[0:600, :], df.iloc[600:800, :], df.iloc[800:1001]

In [16]:
train, dev, test = train_test_dev_split(featurize(df))
X_train, y_train = xy_split(train)
X_dev, y_dev = xy_split(dev)
X_test, y_test = xy_split(test)

In [17]:
def confidence_intervals(accuracy, n, significance_level):
    critical_value=(1-significance_level)/2
    z_alpha=-1*norm.ppf(critical_value)
    se=math.sqrt((accuracy*(1-accuracy))/n)
    return accuracy-(se*z_alpha), accuracy+(se*z_alpha)

In [18]:
def print_results(name, optimal_model):

  # Training accuracy
  print("Training Accuracy: ")
  train_results = optimal_model.predict(X_train)
  print(accuracy_score(y_train, train_results))
  print()

  # Test accuracy
  print("Test Accuracy: ")
  test_results = optimal_model.predict(X_test)
  print(accuracy_score(y_test, test_results))
  accuracy = accuracy_score(y_test, test_results)
  print()

  # Print CI
  print(confidence_intervals(accuracy, len(y_test), .95))


  return train_results, test_results

Baseline: 0.36/0.37

In [19]:
def fit_model(model):
  model.fit(X_train, y_train)
  training_result, test_result = print_results("Logistic Regression", model)
  return training_result, test_result

Used GridSearch and the dev set to hypertune some parameters for Logistic Regression. Was also going to do it for MLP but the runtime was not feasible given the projected time in Colab. Since Random Forest and KNN don't have many parameters to tune, we did those by hand.

In [22]:

C = [float(x) for x in np.linspace(start = 0.01, stop = 0.1, num = 10)]
multi_class = ['multinomial']
solver = ['newton-cg', 'sag', 'saga', 'lbfgs']
class_weight = ['balanced', None]
penalty = ['l2', 'l1']
param_grid = {'C': C,
               'multi_class': multi_class,
               'solver': solver,
               'class_weight': class_weight,
               'penalty': penalty}

opt_log = GridSearch(model = LogisticRegression(random_state = 8), param_grid = param_grid)
opt_log.fit(X_train, y_train, X_dev, y_dev)
print('Test Score for Optimized Parameters:', opt_log.score(X_test, y_test))

100%|██████████| 160/160 [07:24<00:00,  2.78s/it]

Test Score for Optimized Parameters: 0.3781094527363184


In [23]:
opt_log

GridSearch(model=LogisticRegression(C=0.04000000000000001,
                                    multi_class='multinomial', random_state=0,
                                    solver='newton-cg'),
           num_threads=2,
           param_grid={'C': [0.01, 0.020000000000000004, 0.030000000000000006,
                             0.04000000000000001, 0.05000000000000001,
                             0.06000000000000001, 0.07, 0.08,
                             0.09000000000000001, 0.1],
                       'class_weight': ['balanced', None],
                       'multi_class': ['multinomial'], 'penalty': ['l2', 'l1'],
                       'solver': ['newton-cg', 'sag', 'saga', 'lbfgs']})

In [36]:
model_log = LogisticRegression(penalty="l2", C=0.08)
training_result_log, test_result_log = fit_model(model_log)

Training Accuracy: 
0.6466666666666666

Test Accuracy: 
0.417910447761194

(0.34972580353301363, 0.4860950919893744)


In [39]:
model_rf = RandomForestClassifier()
training_result_rf, test_result_rf = fit_model(model_rf)

Training Accuracy: 
1.0

Test Accuracy: 
0.4228855721393035

(0.35459001341043656, 0.49118113086817045)


In [24]:
hidden_layer_sizes = [(x,) for x in np.arange(0, 210, 40)]
activation = ['tanh', 'relu', 'logistic']
alpha = [0.000001, 0.0001, 0.001, 0.05]
learning_rate = ['constant','adaptive', 'invscaling']
solver = ['sgd', 'adam', 'lbfgs']
param_grid = {
    'hidden_layer_sizes': hidden_layer_sizes,
    'activation': activation,
    'solver': solver,
    'alpha': alpha,
    'learning_rate': learning_rate,
}


opt_mlp = GridSearch(model = MLPClassifier(random_state=8), param_grid = param_grid)
opt_mlp.fit(X_train, y_train, X_dev, y_dev)
print('Test Score for Optimized Parameters:', opt_mlp.score(X_test, y_test))

 12%|█▏        | 80/648 [20:59<2:29:02, 15.74s/it]Process NoDaemonPoolWorker-5:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.7/dist-packages/hypopt/model_selection.py", line 145, in _run_thread_job
    model.fit(X_train, y_train)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py", line 752, in fit
    return self._fit(X, y, incremental=False)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py", line 441, in _fit
    X, y, activations, deltas, coef_grads, intercept_grads, layer_units
  File "/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_mu

KeyboardInterrupt: ignored

In [33]:
model_mlp = MLPClassifier(solver="sgd", learning_rate="constant", hidden_layer_sizes=(100,))
training_result_mlp, test_result_mlp = fit_model(model_mlp)

Training Accuracy: 
0.4033333333333333

Test Accuracy: 
0.3582089552238806

(0.29192397847860946, 0.4244939319691517)


In [34]:
model_knn = KNeighborsClassifier(n_neighbors=26)
training_result_knn, test_result_knn = fit_model(model_knn)

Training Accuracy: 
0.42833333333333334

Test Accuracy: 
0.4079601990049751

(0.3400188115073607, 0.4759015865025895)


An ensemble of our previous models as our final model, with a soft voting scheme. We achieved 43.3%, an improvement over the 36/37% baseline on our data.

In [47]:
estimators=[('log', model_log), ('rf', model_rf)]
ensemble = VotingClassifier(estimators, voting='soft')
training_result_en, test_result_en = fit_model(ensemble)

Training Accuracy: 
0.9883333333333333

Test Accuracy: 
0.43283582089552236

(0.3643396874066609, 0.5013319543843838)
